In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import networkx as nx

#Defining Intrinsic Dynamics
def Intrinsic(X):                       # X is basically inintial conditions (No*Nv matrix)
    x = X.T[0]
    y = X.T[1]
    z = X.T[2]

    #equation for the rossler oscillators
    dx = -w*y-z #ODE
    dy = w*x+a*y
    dz = b+z*(x-c)
    state = np.array([dx,dy,dz])
    return state
 #shape of output = Nv*No




#Defining Coupling Dynamics
def Coupling(X): #X is the set of all initial conditions
    S = np.zeros([Nv,No]) #the coupling element

    x_i, x_j = np.meshgrid(X.T[0],X.T[0]) #matrix of only x values at time t
    # y_i, y_j = np.meshgrid(X.T[1],X.T[1])
    # z_i, z_j = np.meshgrid(X.T[2],X.T[2])
    
    Ax = np.multiply(A,(x_i-x_j))
    # Ay = np.multiply(A,(y_i-y_j))
    # Az = np.multiply(A,(z_i-z_j))
    
    coupled_x = Ax.sum(axis=1).T
    # coupled_y = Ay.sum(axis=1).T
    # coupled_z = Az.sum(axis=1).T
    
    S[0,:] = coupled_x
    # S[1,:] = coupled_y
    # S[2,:] = coupled_z
    
    return S.T
#shape of output : No*Nv


#Defining Total Dynamics
def total_dynamics(X,t):                # X is a one dimensional array with No*Nv entries
    X = X.reshape(No,Nv)
    intrinsic_dynamics = Intrinsic(X).T
    coupled_dynamics = Coupling(X)
    X_final = intrinsic_dynamics +  e * np.multiply(G,coupled_dynamics)
    return(X_final.flatten())

#Visualising the Network
def see_graphs(X):  
    plt.figure(figsize=(5,5))                   # X is a nx graph
    pos = nx.circular_layout(X)
    nx.draw(X, pos,node_size = 3,node_color = "red")
    nx.draw_networkx_labels(X, pos)
    plt.show()

def plot_results(t_values,X_final,ts_plots = False):
    plt.figure().set_size_inches(6,8)
    plt.suptitle("Alpha: "+str(e))
    #plt.subplot(131)
    plt.title('X')
    plt.imshow(X_final[-6000::10,::3],origin='lower',aspect='auto')
    plt.ylabel('time')
    plt.xlabel('oscillator number')
    plt.colorbar();plt.show()
    return None
    # plt.subplot(132)
    # plt.title('Y')   
    # plt.imshow(X_final[-6000::10,1::3],origin='lower',aspect='auto')
    # plt.ylabel('time')
    # plt.xlabel('oscillator number')
    # plt.subplot(133)
    # plt.title('Z')   
    # plt.imshow(X_final[-6000::10,2::3],origin='lower',aspect='auto')
    # plt.ylabel('time')
    # plt.xlabel('oscillator number')
    # plt.colorbar()
    #return None

def plot_timeseries(t_values,X_final):
    plt.figure(figsize=(9,4))
    plt.suptitle("Alpha: "+str(e))
    #plt.subplot(211)
    for i in range(0,No):
        plt.plot(t_values[-2000:],X_final[-2000:,3*i],linewidth=0.8)
        plt.xlabel("time")
        plt.ylabel("X")
        plt.show
    # plt.subplot(212)
    # for i in range(0,No):
    #     plt.plot(t_values[-2000:],X_final[-2000:,3*i+1],linewidth=0.8)
    #     plt.xlabel("time")
    #     plt.ylabel("Y")
    # plt.subplot(313)
    # for i in range(0,No):
    #     plt.plot(t_values[-2000:],X_final[-2000:,3*i+2],linewidth=0.8)
    #     plt.xlabel("time")
    #     plt.ylabel("Z")
    return None

def plot_phasespace(X_final):
    plt.figure(figsize=(4,4))
    plt.title("Alpha: "+str(e))
    for osc in range(0,No):
        plt.plot(X_final[-2000:,3*osc],X_final[-2000:,3*osc+1],linewidth=0.8)
        plt.xlabel("X")
        plt.ylabel("Y")
    return None

#specifying the graph
N = 100
graph = nx.scale_free_graph(N) #scale free directed network is created
graph = graph.to_undirected() # changed to undirected graph
nx.draw(graph,node_color='#8F00FF', node_size=20) #to draw
#see_graphs(graph)



#A = np.array([[0,1],[1,0]])
A = nx.to_numpy_matrix(graph)
time = np.linspace(0,1500,30000)

#1)Parameters
No,Nv = 100,3 #No= no of oscillators, Nv= No of variables
a,b,c,w = 0.165,0.2,10,np.random.normal(0.67,0.15,size = 100)#0.67*np.ones(100)67*np.ones(100)#
#delw = np.linspace(-0.15,0.15,No)
#w = w + delw
G = np.array([1,0,0])

e = 0.15 #coupling parameter

X_ini = np.random.rand(No*Nv)
output1 = odeint(total_dynamics, X_ini, time,hmin=0.000001)
print(output1.shape)

p = plot_results(time,output1)

